In [1]:
import os, sys, re, argparse, pdb, glob, pprint, json
import TS_pipe as tsp
import pandas as pd
f***REMOVED*** rdkit import Chem
f***REMOVED*** rdkit.Chem import rdmolfiles
f***REMOVED*** openbabel import pybel
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 200)
json_file = '../results/nbds_b3lyp.json'
df = pd.read_json(json_file, orient='index')
df = df.reset_index(names='Molecule')

In [2]:
#df = df[df['TS guess key value'] < 160.0]
#df = df.reset_index()
df

,Molecule,TS mode,key parameter atoms,TS guess key parameter,TS guess key value,pysis input conformers number,pysis output conformers,pysis output conformers number,pysis conformers properties,orca input conformers number,orca output conformers,orca output conformers number,orca conformers properties
0,C12H9NS,C1C=CCC=C1,"[3, 2, 1, 13, 10, 4]",distances C-C,"[2.257664591553089, 1.590449510468758]",2,"[C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz]",2,"{'C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': [2.259098276493342, 1.5877852151031822]}, 'C12H9NS/8_C12H9NS_filtered_TS_irc/cregened_c...",2,"[C12H9NS/9_C12H9NS_orca/cregened_conformers_0_reopt, C12H9NS/9_C12H9NS_orca/cregened_conformers_1_reopt]",2,"{'C12H9NS/9_C12H9NS_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -915.09535512, 'orca TS E corr': 0.1326328, 'orca TS def2-tzvp full energy': -915.227988313054, 'orca reac1 Gibbs en..."


In [3]:
failed_orca_count = 0
failed_orca_list = []
conformers_by_orca_count = df['pysis output conformers number'].sum()
conformers_to_orca_count = df['orca input conformers number'].sum()
energies_dict = {}
for i in df.index:
    closed_Es = []
    TS_Es = []
    closed_Hs = []
    open_Hs = []
    for j, conf in enumerate(df.loc[i]['orca output conformers']):
        try:
            if not list(df.loc[i]['orca conformers properties'][conf].values()):
                failed_orca_count += 1
                failed_orca_list.append(conf)
            if all(df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value'][j] > 1.8 for j in [0,1]):
                closed_energy = df.loc[i]['orca conformers properties'][conf]['orca reac1 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac1 E corr']
                closed_H = df.loc[i]['orca conformers properties'][conf]['orca reac1 enthalpy']
                open_H = df.loc[i]['orca conformers properties'][conf]['orca reac2 enthalpy']
                print('Structure with ', df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value'], 'selected as metastable')
            else:
                closed_energy = df.loc[i]['orca conformers properties'][conf]['orca reac2 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac2 E corr']
                closed_H = df.loc[i]['orca conformers properties'][conf]['orca reac2 enthalpy']
                open_H = df.loc[i]['orca conformers properties'][conf]['orca reac1 enthalpy']
                print('Structure with ', df.loc[i]['orca conformers properties'][conf]['orca reac2 conformer key value'], 'selected as metastable')
            TS_energy =  df.loc[i]['orca conformers properties'][conf]['orca TS def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca TS E corr']
        except KeyError as e:
            print(f'Conformer {conf} is missing')
            print(e)
            continue
        closed_Es.append(closed_energy)
        TS_Es.append(TS_energy)
        closed_Hs.append(closed_H)
        open_Hs.append(open_H)
    if not ( closed_Es and TS_Es ):
        continue
    energies_dict[f"{df.loc[i]['Molecule']}"] = {}
    energies_dict[f"{df.loc[i]['Molecule']}"]['Closed energies'] = closed_Es
    energies_dict[f"{df.loc[i]['Molecule']}"]['TS energies'] = TS_Es
    energies_dict[f"{df.loc[i]['Molecule']}"]['open enthalpy'] = open_Hs
    energies_dict[f"{df.loc[i]['Molecule']}"]['closed enthalpy'] = closed_Hs
print(f'Conformers generated by ORCA stage: {conformers_to_orca_count}')
print(f'Failed ORCA jobs: {failed_orca_count}')
print(f'Failed ORCA list:', failed_orca_list)
pprint.pprint(energies_dict)
print(len(energies_dict.keys()))

with open('nbs_b3lyp_final_energies.json', 'w') as json_file:
    json.dump(energies_dict, json_file, indent=4)



Structure with  [2.482660459891378, 2.479064862441062] selected as metastable
Conformer C12H9NS/9_C12H9NS_orca/cregened_conformers_1_reopt is missing
'orca reac1 conformer key value'
Conformers generated by ORCA stage: 2
Failed ORCA jobs: 1
Failed ORCA list: ['C12H9NS/9_C12H9NS_orca/cregened_conformers_1_reopt']
{'C12H9NS': {'Closed energies': [-915.167393174497],
             'TS energies': [-915.095355513054],
             'closed enthalpy': [-915.11855337],
             'open enthalpy': [-915.07215705]}}
1


In [4]:
os.getcwd()

'C:\\Users\\***REMOVED***pe672\\Documents\\GitHub\\TS_pipeline\\src'